In [1]:
#初期設定
import os
import pandas as pd
import numpy as np
import time
import lxml
import re
import requests
from bs4 import BeautifulSoup as bs4
import tqdm
import sys
import inspect
# chromeのwebdriver自動更新用
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.options import Options


In [2]:
pd.set_option('display.max_columns', 20)

In [3]:
# https://web-kiwami.com/python-beautyfulsoup4.html
# http://kondou.com/BS4/
# bs4参考

In [4]:
# 1
# all_year
this_year = 2022 #年変わったら変更
get_year = list(range(2011,this_year+1))
df_scraping_companies = pd.DataFrame()
dict1 = {}
for year in get_year:
    if 2011 <= year <= 2017:
        print(year, "出力中")
        # 単年のデータ取得2011~2017
        url =  rf"https://www.ipokiso.com/company/{year}.html"
        html_res = requests.get(url)
        # ページアクセスエラーの出力
        if html_res.status_code != 200:
            print("requests.getでのurlのアクセスができていません")
            lineno = inspect.currentframe().f_lineno
            print(f"エラーが発生しました。行番号: {lineno}")
            sys.exit()
        soup = bs4(html_res.content, 'html.parser')
        find_all_list = soup.find_all(href=re.compile("company/[0-9]{4}/"))
        url_dict = {
            find_all_list[i].text:"https://www.ipokiso.com/" + find_all_list[i]["href"]
            for i in range(len(find_all_list))
        }
        dict1.update(url_dict)
        dfs_list = pd.read_html(url)
        time.sleep(np.random.randint(100,120)/100)
        for i in range(len(dfs_list)):
            dfs_list[i]["上場年"] = f"{year}"
            df_scraping_companies = pd.concat([df_scraping_companies, dfs_list[i]])
            df_scraping_companies = df_scraping_companies.reset_index(drop=True)
        # df_scraping_companies.drop(df_scraping_companies[df_scraping_companies['企業名'] == "企業名"].index , inplace=True)
        # df_scraping_companies.drop(df_scraping_companies[df_scraping_companies['初値'] == "初値"].index , inplace=True)
    else:
        # 単年のデータ取得2018~2022
        print(year, "出力中")
        url = rf"https://www.ipokiso.com/company/{year}.html"
        if year == 2022:
            url = r"https://www.ipokiso.com/company/index.html"
        html_res = requests.get(url)
        # ページアクセスエラーの出力
        if html_res.status_code != 200:
            print("requests.getでのurlのアクセスができていません")
            lineno = inspect.currentframe().f_lineno
            print(f"エラーが発生しました。行番号: {lineno}")
            sys.exit()
        soup = bs4(html_res.content, 'html.parser')
        find_all_list = soup.find_all(href=re.compile("company/[0-9]{4}/"))
        url_dict = {
            find_all_list[i].text:"https://www.ipokiso.com/" + find_all_list[i]["href"]
            for i in range(len(find_all_list))
        }
        dict1.update(url_dict)
        dfs_list = pd.read_html(url)
        # サイト更新時にページのtableのデザインが変更していないかチェック
        if year >= 2022:
            try:
                if dfs_list[0].columns[0] != '企業名':
                    raise ValueError("銘柄一覧から取得したtableの値が「企業名」ではありません")
                elif dfs_list[1].columns[0] != '総合評価':
                    raise ValueError("銘柄一覧から取得したtableの値が「総合評価」ではありません")
            except ValueError as e:
                # エラーが発生した場合の処理
                print("Error: {}".format(e))
                lineno = inspect.currentframe().f_lineno
                print(f"エラーが発生した行番号: {lineno}")   
        time.sleep(np.random.randint(100,120)/100)
        for i in range(0, len(dfs_list) , 2):
            dfs_list_con = pd.concat([dfs_list[i], dfs_list[i+1]], axis=1)
            dfs_list_con["上場年"] = f"{year}"
            if year == 2019:
                dfs_list_con = dfs_list_con.rename(columns={'上場 市場': '上場市場'})
            if year >= 2020:
                dfs_list_con = dfs_list_con.rename(columns={'申し込み期間': '申し込み 期間', '初値上昇率': '初値 上昇率'})
            df_scraping_companies = pd.concat([df_scraping_companies, dfs_list_con],ignore_index=True)
            df_scraping_companies = df_scraping_companies.reset_index(drop=True)
        df_scraping_companies.drop(df_scraping_companies[df_scraping_companies['企業名'] == "企業名"].index , inplace=True)
        df_scraping_companies.drop(df_scraping_companies[df_scraping_companies['初値'] == "初値"].index , inplace=True)
    if year != this_year:
        time.sleep(np.random.randint(100,120)/100)
#codeカラム作成
df_scraping_companies_code = df_scraping_companies["企業名"].str.extract(r'([0-9]{4})').fillna(0).astype(int)
df_scraping_companies["code"] = df_scraping_companies["銘柄 コード"].fillna(df_scraping_companies_code[0])
df_scraping_companies["code"] = df_scraping_companies["code"].astype(int)
# サイト側の入力間違えしているcodeを手動で訂正する 行指定でindex番号はずれる可能性があるため使わない
df_scraping_companies.loc[df_scraping_companies['企業名'] == "シグマクシス", 'code'] = 6088 #シグマクシス
df_scraping_companies.loc[df_scraping_companies['企業名'] == "アビスト 当選！", 'code'] = 6087 #アビスト
df_scraping_companies.loc[df_scraping_companies['企業名'] == "JESCO ホールディングス", 'code'] = 1434 #JESCO
df_scraping_companies.loc[df_scraping_companies['企業名'] == "じげん 当選！", 'code'] = 3679 #じげん

2011 出力中
2012 出力中
2013 出力中
2014 出力中
2015 出力中
2016 出力中
2017 出力中
2018 出力中
2019 出力中
2020 出力中
2021 出力中
2022 出力中


In [5]:
# dict作成
# 空白削除
del dict1[" "]
del dict1[""]
del dict1["\n"]
# dictをdf化
df_scraping_url = pd.DataFrame.from_dict(dict1, orient='index')
# カラム名変更
df_scraping_url = df_scraping_url.rename(columns={0:"url"})
# 間違えているurlを変更
df_scraping_url.url[df_scraping_url.url == "https://www.ipokiso.com/https://www.ipokiso.com/company/2013/zigexn.html"] = "https://www.ipokiso.com/company/2013/zigexn.html"
df_scraping_url = df_scraping_url.rename_axis('index').reset_index()
df_scraping_url = df_scraping_url.rename(columns={'index':'company_name'})

In [6]:
# 2
# seleniumでのデータ取得用ループ　評価
# chromeのwebdriverのinstallしブラウザを起動する 
options = Options()
options.add_argument('--headless')
driver = webdriver.Chrome(ChromeDriverManager().install(),chrome_options=options)
def make_company_info(url):
    val_list = []
    driver.get(url)
    #seleniumの場合ここでsleepしないと正しく取得出来ない場合がある
    time.sleep(np.random.randint(100,120)/100)
    # HTMLを文字コードをUTF-8に変換してから取得します。
    html_res = driver.page_source.encode('utf-8')
#ここでシステムエラー処理する？
    # if html_res.status_code != 200:
    #     print("requests.getでのurlのアクセスができていません")
    #     lineno = inspect.currentframe().f_lineno
    #     print(f"エラーが発生しました。行番号: {lineno}")
    #     sys.exit()
    soup = bs4(html_res, 'html.parser')
    # 全企業codeの取得
    try:
        code = int(re.search(r"[0-9]{4}", soup.title.text).group())
    except AttributeError:
        f = soup.find_all('h1',text=re.compile(r"[0-9]{4}"))[0].text
        code = int(re.search(r"[0-9]{4}", f).group())
    # 3171のtitleのcodeが3172と誤字のための修正
    if code == 3172:
        f = soup.find_all('h1',text=re.compile(r"[0-9]{4}"))[0].text
        code = int(re.search(r"[0-9]{4}", f).group())
    val_list.append(code)
    # 成長性等の評価取得
    table = soup.find('table',class_="company01")
    val = table.find_all('td')
    # valからカラムの値に入れたい◎等の値のみ抽出
    try:
        for i in range(4):
            if ">？<" in str(val[i]):
                keyword = "？"
            elif "/sannkaku02.gif" in str(val[i]):
                keyword = "△"
            elif "/sannkaku.gif" in str(val[i]):
                keyword = "△"
            elif "/sankaku.gif" in str(val[i]):
                keyword = "△"
            elif "/maru02.gif" in str(val[i]):
                keyword = "〇"
            elif "/maru.gif" in str(val[i]):
                keyword = "〇"
            elif "/2maru.gif" in str(val[i]):
                keyword = "◎"
            elif "/s.gif" in str(val[i]):
                keyword = "S"
            elif "/a.gif" in str(val[i]):
                keyword = "A"
            elif "/b.gif" in str(val[i]):
                keyword = "B"
            elif "/c.gif" in str(val[i]):
                keyword = "C"
            elif "/d.gif" in str(val[i]):
                keyword = "D"
            val_list.append(keyword)
    except AttributeError:
        val_list.extend([np.nan,np.nan,np.nan,np.nan])
        print(f"成長性listのエラー{code}")
    print(code)
    # アンケート調査結果の取得 チェック用
    # print(len(soup.find_all("div",class_="vote-bar")))
    # 6行の場合
    if len(soup.find_all("div",class_="vote-bar")) == 6:
        vote_list = []
        number_of_votes_list = soup.find_all("div",class_="vote-bar")
        for value in number_of_votes_list:
            value = int(re.search(r"\"[0-9]+\"",str(value)).group().strip('"'))
            val_list.append(value)
    # 5行の場合
    elif len(soup.find_all("div",class_="vote-bar")) == 5:
        vote_list = []
        val_list.append(np.nan)
        number_of_votes_list = soup.find_all("div",class_="vote-bar")
        for value in number_of_votes_list:
            value = int(re.search(r"\"[0-9]+\"",str(value)).group().strip('"'))
            val_list.append(value)
    # 10行の場合 5行のアンケートがなぜか２つ表示されている銘柄
    elif len(soup.find_all("div",class_="vote-bar")) == 10:
        vote_list = []
        val_list.append(np.nan)
        number_of_votes_list = soup.find_all("div",class_="vote-bar")
        number_of_votes_list = number_of_votes_list[0:5]
        for value in number_of_votes_list:
            value = int(re.search(r"\"[0-9]+\"",str(value)).group().strip('"'))
            val_list.append(value)
    # アンケートがない場合
    elif len(soup.find_all("div",class_="vote-bar")) == 0:
        val_list.extend([np.nan,np.nan,np.nan,np.nan,np.nan,np.nan])
    # その他の行数エラー検知
    else:
        print("票の価格帯の数が分岐にありません")
        driver.quit()
        raise Exception("Error: アンケートの価格帯の数がifの分岐にありません")
    # 今まで取得した情報をリストに格納
    evaluation_list.append(val_list)
df_scraping_evaluation = df_scraping_url.copy()
url_list = df_scraping_url.url.values
evaluation_list = []
[make_company_info(url) for url in url_list[0:]] #ここで検証のurl数変更！
print(evaluation_list)
df_company_01 = pd.DataFrame(evaluation_list,columns=[
    "code",
    "成長性",
    "割安性",
    "話題性",
    "総合評価",
    "初値予想_+200%以上",
    "初値予想_+100%以上+200%未満",
    "初値予想_+50%以上+100%未満",
    "初値予想_+20%以上+50%未満",
    "初値予想_+0%以上+20%未満",
    "初値予想_+0%未満",
])
df_scraping_evaluation[df_company_01.columns] = df_company_01
print(df_scraping_evaluation)
driver.quit()

C:\Users\xxp2p\AppData\Local\Temp/ipykernel_20384/2652847698.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(),chrome_options=options)
C:\Users\xxp2p\AppData\Local\Temp/ipykernel_20384/2652847698.py:6: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(),chrome_options=options)


3170
3169
9206
4977
4583
6055
6619
3659
3277
6054
1383
3444
3657
7777
3276
4582
3656
3655
6052
3654
3653
4579
6626
1909
2882
3275
3652
3168
3649
6051
3274
7150
2229
3648
3646
6072
3261
2931
6165
3179
3669
7164
3178
3668
4929
3667
4585
3666
6071
4584
3280
6070
3177
3896
3176
6069
9386
6067
3175
9201
6065
3665
9419
6064
3664
6618
3174
6063
2930
6062
6061
6060
6059
3662
6058
7157
3661
3173
3660
3172
3171
6090
4245
6089
7167
6086
6088
3294
6087
6085
3293
1429
3682
6084
3681
3680
4588
6082
6081
3679
3678
6080
3189
3677
6079
6078
5698
3288
3187
6077
6076
3186
3185
6075
2587
3184
6074
4978
6248
4587
3674
1420
1419
3673
6073
7215
2588
9025
3672
3182
3671
3670
3180
3181
4586
6034
6033
3904
7815
3905
3221
3199
6032
9517
6031
9551
6030
3903
2883
4248
7175
2586
3198
9418
3901
3902
6029
6028
3900
6026
7816
3452
6027
3698
6025
3697
6099
9467
3694
3696
3695
6098
3197
6240
3196
3692
4591
3195
3300
3691
3690
7172
6097
3689
3193
3688
6096
6095
4247
4979
6094
3299
7169
3297
3192
9024
3687
3191
9090
6616


In [7]:
# 3
def make_company_info_2(url):
    dfs_table_list = pd.read_html(url)
    print(dfs_table_list[1].iloc[0,1])
    time.sleep(np.random.randint(100,120)/100)
    # サイト更新時にデザイン変更がないか検知する（現状ワシントンホテルのみ違うためスルー）
    # 回避urlリスト
    avoidance_url_list = [
        "https://www.ipokiso.com//company/2019/washingtonhotel.html",#[0]に１つ余分に優待情報のtableある
        "https://www.ipokiso.com//company/2020/gmo-fg.html",#table[6][7]が[7][8]にずれている
        "https://www.ipokiso.com//company/2021/frontier.html",#table[6]が無い
        "https://www.ipokiso.com//company/2021/geolocation.html",#table[6]が無い
        "https://www.ipokiso.com//company/2015/gmo-media.html",#table[6][7]が[7][8]にずれている
        ]
    if url not in avoidance_url_list:
        try:
            if dfs_table_list[1].iloc[0,0] != "会社名":
                raise ValueError("銘柄一覧から取得したtableの値が「会社名」ではありません")
            elif dfs_table_list[2].iloc[0,0] != '想定価格':
                raise ValueError("銘柄一覧から取得したtableの値が「想定価格」ではありません")               
            elif dfs_table_list[3].iloc[0,0] != '抽選申込期間':
                raise ValueError("銘柄一覧から取得したtableの値が「抽選申込期間」ではありません")
            elif not '公募' in dfs_table_list[4].iloc[0,0]:
                raise ValueError("銘柄一覧から取得したtableの値が「公募株数等」ではありません")
            elif dfs_table_list[5].iloc[0,1] != '証券会社名':
                raise ValueError("銘柄一覧から取得したtableの値が「証券会社名」ではありません")
        except ValueError as e:
            # エラーが発生した場合の処理
            print("Error: {}".format(e))
            lineno = inspect.currentframe().f_lineno
            print(f"エラーが発生した行番号: {lineno}")
            print(f"エラーが発生したURL:{url}")
            print(dfs_table_list)
            sys.exit()
        # 会社名にREITと投資法人という文字列が無いことを判定する
        if not ("REIT" in dfs_table_list[1].iloc[0,1]) and not ("投資法人" in dfs_table_list[1].iloc[0,1]):
            try:
                if dfs_table_list[6].columns[0] != '株主名':
                    raise ValueError("銘柄一覧から取得したtableの値が「株主名」ではありません")
                elif not ("（百万円）" in dfs_table_list[7].iloc[1,0]) and not ("（千米ドル）" in dfs_table_list[7].iloc[1,0]):
                    raise ValueError("銘柄一覧から取得したtableの値が「財務データ等」ではありません")
            except ValueError as e:
                # エラーが発生した場合の処理
                print("Error: {}".format(e))
                lineno = inspect.currentframe().f_lineno
                print(f"エラーが発生した行番号: {lineno}")
                print(f"エラーが発生したURL:{url}")
                print(dfs_table_list)
                sys.exit()

    # ここからdfsの結合、整形コード
    df_con_list = []
    for i in range(len(dfs_table_list)):
        # 基本情報
        if i == 1:
            # tableの位置指定
            df_table = dfs_table_list[i]
            # table位置が違うものの例外処理
            if df_table.iloc[0,0] != "会社名":
                df_table = dfs_table_list[i + 1]
                print(url)
            # df整形：基本情報
            cols = df_table.T.values[0]
            val = df_table.T.values[1]
            # 追加する1件分の基本情報のdf作成
            df_table_1 = pd.DataFrame([val],columns=cols)
            df_con_list.append(df_table_1)
            # 表記の揺れ統一のための修正

        # IPO日程と価格決定（初値予想）
        if i == 2:
            # tableの位置指定
            df_table = dfs_table_list[i]
            # table位置が違うものの例外処理
            if dfs_table_list[2].iloc[0,0] != '想定価格':
                df_table = dfs_table_list[i + 1]
                print(url)
            # df整形：IPO日程と価格決定（初値予想）
            cols = df_table.T.values[0]
            val = df_table.T.values[1]
            df_table_2 = pd.DataFrame([val],columns=cols)
            df_con_list.append(df_table_2)
            # 表記の揺れ統一のための修正
            
        # IPOスケジュール
        if i == 3:
            # tableの位置指定 # tableの位置指定
            df_table = dfs_table_list[i]
             # table位置が違うものの例外処理
            if dfs_table_list[3].iloc[0,0] != '抽選申込期間':
                df_table = dfs_table_list[i + 1]
                print(url)
            
            # df整形：IPOスケジュール
            cols = df_table.T.values[0]
            val = df_table.T.values[1]
            df_table_3 = pd.DataFrame([val],columns=cols)
            df_con_list.append(df_table_3)
            # 表記の揺れ統一のための修正
        # IPO当選株数
        if i == 4:
            # tableの位置指定
            df_table = dfs_table_list[i]
             # table位置が違うものの例外処理
            if not '公募' in dfs_table_list[4].iloc[0,0]:
                df_table = dfs_table_list[i + 1]
                print(url)
            # ワシントンホテルのみ[1]に優待情報があるためtableをずらしている
            if url == "https://www.ipokiso.com//company/2019/washingtonhotel.html":
                df_table = dfs_table_list[i + 1]
            
            # df整形：IPO当選株数
            cols = df_table.T.values[0]
            val = df_table.T.values[1]
            df_table_4 = pd.DataFrame([val],columns=cols)
            df_con_list.append(df_table_4)
            # 表記の揺れ統一のための修正

        # 幹事証券リスト（管理人独自予想あり）
        if i == 5:
            # tableの位置指定
            df_table = dfs_table_list[i]
             # table位置が違うものの例外処理
            if dfs_table_list[5].iloc[0,1] != '証券会社名':
                df_table = dfs_table_list[i + 1]
                print(url)
            # df整形：幹事証券リスト（管理人独自予想あり）
            df_table_5 = pd.DataFrame()
            for j in range(1,len(df_table.iloc[:,0])):
                val = df_table.iloc[j,:].values
                cols = df_table.iloc[0,:]
                cols[0] = "幹事種類"
                cols = [f"{col}_{j}" for col in cols]
                df_add = pd.DataFrame([val],columns=cols)
                df_table_5 = pd.concat([df_table_5, df_add], axis=1)
            df_con_list.append(df_table_5)

        # 株主構成、ロックアップなど
        if i == 6:
            if dfs_table_list[1].iloc[0,0] == "会社名":
                company_name = dfs_table_list[1].iloc[0,1]
            elif dfs_table_list[2].iloc[0,0] == "会社名":
                company_name = dfs_table_list[2].iloc[0,1]
            if not ("REIT" in company_name or "投資法人" in company_name or  ("福証" in company_name and not "東証" in company_name)):
                # tableの位置指定
                df_table = dfs_table_list[i]
                # table位置が違うものの例外処理
                if dfs_table_list[6].columns[0] != '株主名':
                    df_table = dfs_table_list[i + 1]
                    print(url)
                df_table_6 = pd.DataFrame()
                for k in range(0,len(df_table.iloc[:,0])):
                    val = df_table.iloc[k,:].values
                    cols = df_table.columns
                    cols = [f"{col}_{k+1}" for col in cols]
                    df = pd.DataFrame([val],columns=cols)
                    df_table_6 = pd.concat([df_table_6, df], axis=1)
                df_con_list.append(df_table_6)

            # 企業業績のデータ（5年分）
        if i == 7:
            if not ("REIT" in company_name or "投資法人" in company_name):

                # tableの位置指定
                df_table = dfs_table_list[i]
                # table位置が違うものの例外処理
                if not ("（百万円）" in dfs_table_list[7].iloc[1,0]) and not ("（千米ドル）" in dfs_table_list[7].iloc[1,0]):
                    df_table = dfs_table_list[i + 1]
                    print(url)
                
                df_table_7 = pd.DataFrame()
                for h in range(1,len(df_table.T.iloc[:,0])):
                    val = df_table.T.iloc[h,:].values
                    # カラム名の誤字修正
                    cols = df_table.T.iloc[0,:]
                    cols[0] = "年月"
                    cols = [f"{col}_{h}年目" for col in cols]
                    df = pd.DataFrame([val],columns=cols)
                    df_table_7 = pd.concat([df_table_7, df], axis=1)
                df_con_list.append(df_table_7)


            # 全件のdfに追加していく
    df_table_all = pd.concat(df_con_list,axis=1)
    return df_table_all
df_scraping_table_all = pd.DataFrame()
url_list = df_scraping_url.url.values
for url in url_list[0:]:#ここでurl数変更
    df_table_all = make_company_info_2(url) 
    df_scraping_table_all = pd.concat([df_scraping_table_all,df_table_all],axis=0,ignore_index=True)
print(df_scraping_table_all)

アイセイ薬局（3170）　【JASDAQスタンダード】
ミサワ（3169）　【JASDAQスタンダード】
スターフライヤー（9206）　【東証2部】
新田ゼラチン（4977）　【東証2部】
カイオム・バイオサイエンス（4583）　【東証マザーズ】
ジャパンマテリアル（6055）　【東証2部】
ダブル・スコープ（6619）　【東証マザーズ】
ネクソン（3659）　【東証1部】
サンセイランディック（3277）　【ＪＡＳＤＡＱスタンダード】
リブセンス（6054）　【東証マザーズ】
ベルグアース（1383）　【JASDAQスタンダード】
菊池製作所（3444）　【JASDAQスタンダード】
ポールトゥウィン・ピットクルーホールディングス（3657）　【東証マザーズ】
スリー・ディー・マトリックス（7777）　【JASDAQグロース】
日本管理センター（3276）　【JASDAQスタンダード】
シンバイオ製薬（4582）　【JASDAQグロース】
KLab[クラブ]（3656）　【東証マザーズ】
ブレインパッド（3655）　【東証マザーズ】
イーピーミント（6052）　【JASDAQスタンダード】
ヒト・コミュニケーションズ（3654）　【ＪＡＳＤＡＱスタンダード】
モルフォ（3653）　【東証マザーズ】
ラクオリア創薬（4579）　【JASDAQグロース】
SEMITEC（6626）　【JASDAQスタンダード】
日本ドライケミカル（1909）　【東証2部】
イートアンド（2882）　【JASDAQスタンダード】
ハウスコム（3275）　【JASDAQスタンダード】
ディジタルメディアプロフェッショナル（3652）　【東証マザーズ】
クロタニコーポレーション（3168）　【東証マザーズ】
ファインデックス（3649）　【東証マザーズ】
アイ・アールジャパン（6051）　【JASDAQスタンダード】
アイディホーム（3274）　【JASDAQスタンダード】
島根銀行（7150）　【東証2部】
カルビー（2229）　【東証1部】
AGS（3648）　【東証2部】
駅探（3646）　【東証マザーズ】
地盤ネット（6072）　【東証マザーズ】
グランディーズ（3261）　【福岡Qボード】
ユーグレナ（2931）　【東証マザーズ】
パンチ工業（6165）　【東証2部】
シュッピン

In [8]:
# 時短のためcode含めたcsv読み込み　コード完成後に削除する
# df_scraping_evaluation = pd.read_csv(r"C:\Users\xxp2p\OneDrive\デスクトップ\df_scraping_url.csv", index_col=0)
# df_scraping_evaluation

全スクレイピングのdf作成　df_scraping_all


In [9]:
#  cocat df_scraping_eva_tab作成
df_scraping_eva_tab = pd.concat([df_scraping_evaluation,df_scraping_table_all],axis=1)

In [10]:
# merge df_scraping_all作成 これがスクレイピング全データでこれを元にoutputのdf作る。
df_scraping_all = pd.merge(df_scraping_eva_tab, df_scraping_companies, on="code", how="inner", suffixes=("","_comp"))
df_scraping_all = df_scraping_all.drop_duplicates()

In [11]:
# 上場中止銘柄とサイト側のミスデータの行削除 文字列に中止等があれば削除
df_scraping_all = df_scraping_all[~df_scraping_all['企業名'].str.contains('中止')]
df_scraping_all = df_scraping_all[~df_scraping_all['上場日_comp'].str.contains('中止')]
df_scraping_all = df_scraping_all[~df_scraping_all['company_name'].str.contains('ヤマイチエステート')]#ミスデータ
df_scraping_all = df_scraping_all[~df_scraping_all['company_name'].str.contains('テノ．\r\nホールディングス')]#ミスデータ
df_scraping_all = df_scraping_all[~df_scraping_all['company_name'].str.contains('じげん（3679）\n')]#ミスデータ
df_scraping_all = df_scraping_all.reset_index(drop=True)

In [12]:
# チェック用
df_scraping_all[df_scraping_all["code"]==3679]

,company_name,url,code,成長性,割安性,話題性,総合評価,初値予想_+200%以上,初値予想_+100%以上+200%未満,初値予想_+50%以上+100%未満,...,当たり本数,想定価格_comp,仮条件_comp,公募価格_comp,初値_comp,初値 上昇率,狙い目証券,上場年,総合 評価,総合評価_comp
98,じげん,https://www.ipokiso.com//company/2013/zigexn.html,3679,◎,〇,◎,A,NaN,NaN,NaN,...,"85,500本",500円,500円 ～600円,600円,"1,750円","+1,150円 （192％）",マネックス SBI 東海東京,2013,NaN,NaN


In [22]:
df_scraping_all

,company_name,url,code,成長性,割安性,話題性,総合評価,初値予想_+200%以上,初値予想_+100%以上+200%未満,初値予想_+50%以上+100%未満,...,当たり本数,想定価格_comp,仮条件_comp,公募価格_comp,初値_comp,初値 上昇率,狙い目証券,上場年,総合 評価,総合評価_comp
0,アイセイ薬局,https://www.ipokiso.com//company/2011/aisei.html,3170,◎,〇,〇,B,NaN,NaN,NaN,...,"2,490本","3,460円","3,300円 ～3,460円","3,460円","3,460円",±0円 （0.0％）,SMBC日興 SBI カブドットコム 楽天,2011,NaN,NaN
1,ミサワ,https://www.ipokiso.com//company/2011/unico.html,3169,〇,△,〇,C,NaN,NaN,NaN,...,"6,000本",860円,760円 ～860円,860円,"1,209円",+349円 （40.6％）,SMBC日興 （副） SBI 松井 カブドットコム,2011,NaN,NaN
2,スターフライヤー,https://www.ipokiso.com//company/2011/starflye...,9206,〇,◎,〇,B,NaN,NaN,NaN,...,"2,000本","3,180円","2,900円 ～3,180円","3,180円","5,500円","+2,320円 （73.0％）",SMBC日興 （副） エイチエス マネックス カブドットコム 東海東京,2011,NaN,NaN
3,新田ゼラチン,https://www.ipokiso.com//company/2011/nitta-ge...,4977,△,〇,△,D,NaN,NaN,NaN,...,"36,280本",570円,500円 ～570円,500円,462円,-38円 （-7.6％）,SMBC日興（主） SBI,2011,NaN,NaN
4,カイオム・\r\nバイオサイエンス,https://www.ipokiso.com//company/2011/chiome.html,4583,△,△,△,D,NaN,NaN,NaN,...,"7,770本","2,700円","2,100円 ～2,700円","2,700円","2,480円",-220円 （-8.1％）,SBI 岩井コスモ むさし カブドットコム 楽天,2011,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
986,CaSy,https://www.ipokiso.com//company/2022/casy.html,9215,〇,△,〇,C,253.0,34.0,41.0,...,"2,433本（少ない）","1,350円","1,230円～1,350円","1,350円","2,001円",+651円（+48.2％）,SBI（主） 松井 マネックス 楽天 岩井 ネオトレード DMM株,2022,NaN,NaN
987,エッジテクノロジー,https://www.ipokiso.com//company/2022/edge-tec...,4268,〇,△,〇,D,92.0,46.0,187.0,...,"34,759本（多い）",690円（低価格）,300円～350円,350円,694円,+344円（+98.3％）,SMBC日興（主） 野村（副） SBI 楽天 松井 DMM株,2022,NaN,NaN
988,ライトワークス,https://www.ipokiso.com//company/2022/lightwor...,4267,◎,△,〇,C,116.0,28.0,53.0,...,"3,680本","2,100円","1,900円～2,100円","2,100円","3,000円",+900円（+42.9％）,SMBC日興（副） SBI 楽天 マネックス 岡三オンライン DMM株,2022,NaN,NaN
989,セイファート,https://www.ipokiso.com//company/2022/seyfert....,9213,△,△,〇,D,53.0,16.0,19.0,...,"8,574本","1,120円","1,060円～1,120円","1,120円","1,030円",-90円（-8.0％）,大和（副） 岩井 マネックス CONNECT DMM株,2022,NaN,NaN


次はここから！2022/12/21予定
1,文字整形
2,分割

最終出力dfの4つ作成 
df_assessments
df_schedule
df_underwriters
df_shareholders

In [13]:
# df_scraping_all.to_csv(r"C:\Users\xxp2p\OneDrive\デスクトップ\df_scraping_all.csv", encoding='utf-8_sig',index=False)

In [119]:
# 評価+ 会社設立年 + 市場(G,P,S,…)　+ IPO日程と価格決定(初値予測) + 初値予想アンケート output_data1/4
df_assessments = df_scraping_all[[
    "code",
    "成長性",
    "割安性",
    "話題性",
    "総合評価",
    "初値予想_+200%以上",
    "初値予想_+100%以上+200%未満",
    "初値予想_+50%以上+100%未満",
    "初値予想_+20%以上+50%未満",
    "初値予想_+0%以上+20%未満",
    "初値予想_+0%未満",
    "会社設立",
    "会社名",#市場に変形する
    "想定価格",
    "仮条件",
    "公募価格",
    "初値予想（独自）",
    "初値"
]]
# 会社設立の値を西暦4桁のみに置換(float型)
df_assessments["会社設立"] = df_assessments["会社設立"].str.extract(r'([0-9]{4})').fillna(np.nan).astype(float)
# 分割　（二つに列を分ける場合）
df_assessments = df_assessments.assign(A=df_assessments['仮条件'].str.split('～|-').str[0])
df_assessments = df_assessments.assign(B=df_assessments['仮条件'].str.split('～|-').str[1])
df_assessments = df_assessments.assign(C=df_assessments['初値予想（独自）'].str.split('～').str[0])
df_assessments = df_assessments.assign(D=df_assessments['初値予想（独自）'].str.split('～').str[1])

# 分割　（置換ver）　初値（円）分割して代入
df_assessments["初値"] = df_assessments["初値"].str.split('円').str[0]


# rename
df_assessments = df_assessments.rename(columns={
    "想定価格":"想定価格（円）",
    "仮条件":"仮条件（円）",
    "A":"仮条件（円）_下値",
    "B":"仮条件（円）_上値",
    "公募価格":"公募価格（円）",
    "C":"初値予想（独自）（円）_下値",
    "D":"初値予想（独自）（円）_上値",
    "初値":"初値（円）",
    })
# . ,  円の削除
df_assessments['想定価格（円）'] = df_assessments['想定価格（円）'].str.replace('円','').str.replace(',','').str.replace('.','')
df_assessments['公募価格（円）'] = df_assessments['公募価格（円）'].str.replace('円','').str.replace(',','').str.replace('.','')
df_assessments['初値（円）'] = df_assessments['初値（円）'].str.replace('円','').str.replace(',','').str.replace('.','')
df_assessments['仮条件（円）_下値'] = df_assessments['仮条件（円）_下値'].str.replace('円','').str.replace(',','').str.replace('.','')
df_assessments['仮条件（円）_上値'] = df_assessments['仮条件（円）_上値'].str.replace('円','').str.replace(',','').str.replace('.','')
df_assessments['初値予想（独自）（円）_下値'] = df_assessments['初値予想（独自）（円）_下値'].str.replace('～','').str.replace(',','').str.replace('.','')
df_assessments['初値予想（独自）（円）_上値'] = df_assessments['初値予想（独自）（円）_上値'].str.replace('～','').str.replace(',','').str.replace('.','')
# 列にある文字列を削除して、float型に変える（ついでに空欄がなくなる）
df_assessments['想定価格（円）'] = df_assessments['想定価格（円）'].str.extract(r'(\d+)').astype(float)
df_assessments['公募価格（円）'] = df_assessments['公募価格（円）'].str.extract(r'(\d+)').astype(float)
df_assessments['初値（円）'] = df_assessments['初値（円）'].str.extract(r'(\d+)').astype(float)
df_assessments['仮条件（円）_下値'] = df_assessments['仮条件（円）_下値'].str.extract(r'(\d+)').astype(float)
df_assessments['仮条件（円）_上値'] = df_assessments['仮条件（円）_上値'].str.extract(r'(\d+)').astype(float)
df_assessments['初値予想（独自）（円）_下値'] = df_assessments['初値予想（独自）（円）_下値'].str.extract(r'(\d+)').astype(float)
df_assessments['初値予想（独自）（円）_上値'] = df_assessments['初値予想（独自）（円）_上値'].str.extract(r'(\d+)').astype(float)
# 欠損値をうめる　予想範囲がなく下値のみのものを上値にも同額いれる
df_assessments['仮条件（円）_上値'] = df_assessments['仮条件（円）_上値'].fillna(df_assessments['仮条件（円）_下値'])
df_assessments['初値予想（独自）（円）_上値'] = df_assessments['初値予想（独自）（円）_上値'].fillna(df_assessments['初値予想（独自）（円）_下値'])
# 不必要な列drop
df_assessments = df_assessments.drop("仮条件（円）", axis=1)
df_assessments = df_assessments.drop("初値予想（独自）", axis=1)
print(df_assessments.dtypes)
df_assessments

code                     int64
成長性                     object
割安性                     object
話題性                     object
総合評価                    object
初値予想_+200%以上           float64
初値予想_+100%以上+200%未満    float64
初値予想_+50%以上+100%未満     float64
初値予想_+20%以上+50%未満      float64
初値予想_+0%以上+20%未満       float64
初値予想_+0%未満             float64
会社設立                   float64
会社名                     object
想定価格（円）                float64
公募価格（円）                float64
初値（円）                  float64
仮条件（円）_下値              float64
仮条件（円）_上値              float64
初値予想（独自）（円）_下値         float64
初値予想（独自）（円）_上値         float64
dtype: object


C:\Users\xxp2p\AppData\Local\Temp/ipykernel_20384/647402949.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_assessments["会社設立"] = df_assessments["会社設立"].str.extract(r'([0-9]{4})').fillna(np.nan).astype(float)
C:\Users\xxp2p\AppData\Local\Temp/ipykernel_20384/647402949.py:46: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_assessments['想定価格（円）'] = df_assessments['想定価格（円）'].str.replace('円','').str.replace(',','').str.replace('.','')
C:\Users\xxp2p\AppData\Local\Temp/ipykernel_20384/647402949.py:47: FutureWarning: The default value of regex will change from True to False in a future ve

,code,成長性,割安性,話題性,総合評価,初値予想_+200%以上,初値予想_+100%以上+200%未満,初値予想_+50%以上+100%未満,初値予想_+20%以上+50%未満,初値予想_+0%以上+20%未満,初値予想_+0%未満,会社設立,会社名,想定価格（円）,公募価格（円）,初値（円）,仮条件（円）_下値,仮条件（円）_上値,初値予想（独自）（円）_下値,初値予想（独自）（円）_上値
0,3170,◎,〇,〇,B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,アイセイ薬局（3170） 【JASDAQスタンダード】,3460.0,3460.0,3460.0,3300.0,3460.0,NaN,NaN
1,3169,〇,△,〇,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ミサワ（3169） 【JASDAQスタンダード】,860.0,860.0,1209.0,760.0,860.0,NaN,NaN
2,9206,〇,◎,〇,B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,スターフライヤー（9206） 【東証2部】,3180.0,3180.0,5500.0,2900.0,3180.0,NaN,NaN
3,4977,△,〇,△,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,新田ゼラチン（4977） 【東証2部】,570.0,500.0,462.0,500.0,570.0,NaN,NaN
4,4583,△,△,△,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,カイオム・バイオサイエンス（4583） 【東証マザーズ】,2700.0,2700.0,2480.0,2100.0,2700.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
986,9215,〇,△,〇,C,253.0,34.0,41.0,36.0,86.0,108.0,2014.0,CaSy[カジー]（9215） 【東証マザーズ】,1350.0,1350.0,2001.0,1230.0,1350.0,1350.0,1750.0
987,4268,〇,△,〇,D,92.0,46.0,187.0,102.0,53.0,244.0,2014.0,エッジテクノロジー（4268） 【東証マザーズ】,690.0,350.0,694.0,300.0,350.0,322.0,550.0
988,4267,◎,△,〇,C,116.0,28.0,53.0,59.0,103.0,224.0,1995.0,ライトワークス（4267） 【東証マザーズ】,2100.0,2100.0,3000.0,1900.0,2100.0,2100.0,2900.0
989,9213,△,△,〇,D,53.0,16.0,19.0,40.0,80.0,192.0,1991.0,セイファート（9213） 【JASDAQスタンダード】,1120.0,1120.0,1030.0,1060.0,1120.0,1030.0,1500.0


In [15]:
# IPOスケジュール + IPO当選株数 output2/4
df_schedule = df_scraping_all[[
    "code",
    "抽選申込期間",
    "当選発表日",
    "購入申込期間",
    "上場日",
    "公募株数",
    "売出株数（OA含む）",
    "当選株数合計"
]]
df_schedule

,code,抽選申込期間,当選発表日,購入申込期間,上場日,公募株数,売出株数（OA含む）,当選株数合計
0,3170,12月6日（火）～12月12日（月）,12月13日（火）,12月15日（木）～12月20日（火）,12月22日（木）,"249,000株","37,300株","286,300株"
1,3169,12月7日（水）～12月13日（火）,12月14日（水）,12月15日（木）～12月20日（火）,12月22日（木）,"380,000株","310,000株","690,000株"
2,9206,12月5日（月）～12月9日（金）,12月12日（月）,12月14日（水）～12月19日（月）,12月21日（水）,"140,000株","90,000株","230,000株"
3,4977,12月5日（月）～12月9日（金）,12月12日（月）,12月13日（火）～12月15日（木）,12月20日（火）,"3,000,000株","1,170,000株","4,170,000株"
4,4583,12月2日（金）～12月8日（木）,12月9日（金）,12月13日（火）～12月16日（金）,12月20日（火）,"188,000株","705,500株","893,500株"
...,...,...,...,...,...,...,...,...
986,9215,2月4日(金)～2月10日(木),2月14日(月),2月15日(火)～2月18日(金),2月22日(火),"125,000株","118,300株","243,300株"
987,4268,2月1日(火)～2月7日(月),2月8日(火),2月9日(水)～2月15日(火),2月17日(木),"544,000株","2,862,900株","3,406,900株"
988,4267,1月21日(金)～1月27日(木),1月28日(金),1月31日(月)～2月3日(木),2月9日(水),"160,000株","208,000株","368,000株"
989,9213,1月19日(水)～1月25日(火),1月26日(水),1月27日(木)～2月1日(火),2月4日(金),"420,800株","436,600株","857,400株"


In [16]:
# 幹事証券リスト（管理人独自予想あり）output3/4
df_underwriters = df_scraping_all.filter(regex="code|幹事種類_|証券会社名_|割当率_|割当株数_|当選本数 （枚）_|完全抽選本数 （予想）_")
df_underwriters

,code,幹事種類_1,証券会社名_1,割当率_1,割当株数_1,当選本数 （枚）_1,完全抽選本数 （予想）_1,幹事種類_2,証券会社名_2,割当率_2,...,割当率_102,割当株数_102,当選本数 （枚）_102,完全抽選本数 （予想）_102,幹事種類_103,証券会社名_103,割当率_103,割当株数_103,当選本数 （枚）_103,完全抽選本数 （予想）_103
0,3170,主幹事,大和証券,75.22％,"187,300株","1,873枚",187枚,幹事,SMBC日興証券,14.98％,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3169,主幹事,みずほ インベスターズ 証券,80.00％,"480,000株","4,800枚",480枚,幹事,SMBC日興証券,10.00％,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9206,主幹事,大和証券,75.00％,"150,000株","1,500枚",225枚,幹事,SMBC日興証券,5.00％,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4977,主幹事,SMBC日興証券,74.99％,"2,720,700株","27,207枚","2,720枚",幹事,野村證券,9.00％,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4583,主幹事,大和証券,80.04％,"621,900株","6,219枚",932枚,幹事,みずほ証券,4.00％,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
986,9215,主幹事,SBI証券,86.93％,"211,500株","2,115枚",951枚,幹事,みずほ証券,7.81％,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
987,4268,主幹事,SMBC日興証券,93.05％,"2,899,900株","28,999枚","2,899枚",幹事,野村證券,4.35％,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
988,4267,主幹事,エイチ・エス証券,84.35％,"310,400株","3,104枚",310枚,幹事,SMBC日興証券,5.22％,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
989,9213,主幹事,みずほ証券,95.67％,"820,300株","8,203枚",820枚,幹事,大和証券,1.74％,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
# 株主構成 + ロックアップ[6]　output4/4
df_shareholders = df_scraping_all.filter(regex="code|株主名_|比率_|ロック アップ_|ロック  アップ_")
df_shareholders

,code,株主名_1,比率_1,株主名_2,比率_2,株主名_3,比率_3,株主名_4,比率_4,株主名_5,...,ロック アップ_1,ロック アップ_2,ロック アップ_3,ロック アップ_4,ロック アップ_5,ロック アップ_6,ロック アップ_7,ロック アップ_8,ロック アップ_9,ロック アップ_10
0,3170,岡村 幸彦（社長）,30.41％,（株）望未,18.27％,メディパルホールディングス,18.27％,中央エム・リース,9.14％,SMBCキャピタル８号,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3169,三沢 太（社長）,76.15％,飯塚 智香,7.96％,尾張 睦,6.36％,大塚 幸江,4.09％,従業員持ち株会,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9206,"DCM IV, L.P.",17.26％,ＴＯＴＯ,5.08％,安川電機,3.29％,北九州エアターミナル,2.90％,九州電力,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4977,アイビーピー,25.10％,ニッタ,6.60％,三菱東京ＵＦＪ銀行,4.88％,三井住友銀行,4.71％,（有）以和貴,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4583,ファストトラックイニシアティブ１号,18.89％,藤原 正明（社長）,10.17％,オリンパスビジネスクリエイツ,8.59％,太田 邦史,5.65％,清田 圭一,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
986,9215,加茂 雄一（社長）,21.56％,ワタキューセイモア（株）,19.11％,池田 裕樹,17.75％,胡桃 沢精一,7.96％,（株）I.K.D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
987,4268,住本 幸士（社長）,93.56％,島田 雄太,3.09％,治田 知明,1.20％,坂西 茂,0.39％,生澤 剛士,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
988,4267,（株）エプシモーヴェ,53.21％,江口 夏郎（社長）,15.87％,小迫 宏行,7.33％,ライトワークス従業員持株会,4.62％,松林 洋太,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
989,9213,（株）ビューティープロスペリティー,32.28％,長谷川 高志（社長）,23.24％,長谷川 美栄,9.59％,山田 実,6.91％,藤本 宏志,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


以上本番用

In [118]:
df_assessments.to_csv(r"C:\Users\xxp2p\OneDrive\デスクトップ\df_assessments2.csv", encoding='utf-8_sig',index=False)

In [19]:
# チェック用
df_scraping_all[df_scraping_all['code']==4932]

,company_name,url,code,成長性,割安性,話題性,総合評価,初値予想_+200%以上,初値予想_+100%以上+200%未満,初値予想_+50%以上+100%未満,...,当たり本数,想定価格_comp,仮条件_comp,公募価格_comp,初値_comp,初値 上昇率,狙い目証券,上場年,総合 評価,総合評価_comp
749,アルマード,https://www.ipokiso.com//company/2021/almado.html,4932,△,△,〇,C,77.0,42.0,166.0,...,"60,950本（多い）",840円（低価格）,840円～880円,880円,861円,-19円（-2.2％）,野村（主） SMBC日興 SBI マネックス 楽天 岡三オンライン DMM株,2021,NaN,NaN


In [20]:
# 重複チェック
# df_scraping_all['code'].value_counts(ascending=False)[0:10]

In [21]:
# 時短のためcode含めたcsv読み込み
# df_scraping_evaluation = pd.read_csv("df_scraping_url.csv", index_col=0)
# df_scraping_evaluation